<center><img src="logo.png" alt="Header" style="width: 800px;"/></center>

In [1]:
# 导入库  import library
import cv2
import ipywidgets.widgets as widgets
from IPython.display import display
import logging
import time
import threading
import inspect
import ctypes
import numpy as np
import pyzbar.pyzbar as pyzbar

import random
from enum import Enum, auto

In [2]:
# Dogzilla drive library
from DOGZILLALib import DOGZILLA
g_dog = DOGZILLA()

# 二维码识别 QRCode recognition

In [3]:
# 中文开关，默认为英文 Chinese switch. The default value is English
g_ENABLE_CHINESE = False

Name_widgets = {
    'Close_Camera': ("Close_Camera", "关闭摄像头")
}

In [4]:
# 图像数据转化  Image data transformation
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [5]:
# 关闭线程  stop thread
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [6]:
# 创建摄像头显示组件  Create the camera display component
image_widget = widgets.Image(format='jpeg', width=640, height=480)  

# 打开摄像头，数字0需根据/dev/videoX修改为X
# Turn on the camera, you need to change the number 0 to X based on /dev/videoX
image = cv2.VideoCapture(0)
image.set(3, 640)
image.set(4, 480)
image.set(5, 30)
image.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))

True

In [7]:
# 关闭摄像头 Close_Camera
button_Close_Camera = widgets.Button(  
    value=False,  
    description=Name_widgets['Close_Camera'][g_ENABLE_CHINESE],      
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )


# 按键按下事件处理   Key press event processing
def on_button_close_camera(b):
    if b.description == Name_widgets['Close_Camera'][g_ENABLE_CHINESE]:
        # 停止线程，释放摄像头  Stop the thread and release the camera
        b.icon = 'uncheck'
        stop_thread(thread1)
        image.release()
    
# 关联按键事件回调 Button event callbacks
button_Close_Camera.on_click(on_button_close_camera)

In [9]:
# The list that will hold the directional commands ("right" or "left")
commands_array = []

In [10]:
# 解析图像中的二维码信息  Analyze the QR code information in the image

# Custom-made class so that we are able to effectively use Enums to handle machine states
class MovementState(Enum):
    IDLE = auto()
    WALKING_FORWARD = auto()
    SCANNING = auto()
    SPINNING = auto()
    NEARING_WALL = auto()
    ALIGNING_QR = auto()

class MovementSystem:
    def __init__(self, g_dog, commands_array):
        # We initialize the robot in this state as it allows out to ensure
        # Walk until it finds a QR code before navigation, it's a failsafe so that the current Location is never NONE
        # If the current Location is NONE, the robot will start itself off path and won't execute the first command correctly
        self.g_dog = g_dog
        self.state = MovementState.WALKING_FORWARD
        self.start_time = time.time()
        self.current_node_location = None
        self.previous_node = None
        self.bool_determine_path = True
        self.last_seen_node = None
        self.commands_array = commands_array
        self.target_command = commands_array[0]
        self.adjacent_nodes = set()
        self.completed_maze = False

    def get_state(self):
        _curr_state = ""
        if self.state == MovementState.IDLE:
            return "IDLE"
        elif self.state == MovementState.WALKING_FORWARD:
            return "WALKING_FORWARD"
        elif self.state == MovementState.SCANNING:
            return "SCANNING"
        elif self.state == MovementState.SPINNING:
            return "SPINNING"
        elif self.state == MovementState.NEARING_WALL:
            return "NEARING_WALL"
        elif self.state == MovementState.ALIGNING_QR:
            return "ALIGNING_QR"

    def get_current_location(self):
        return self.current_node_location
    def get_previous_node(self):
        return self.previous_node
    def get_bool_determine_path(self):
        return self.bool_determine_path
    def get_target_command(self):
        return self.target_command
    def get_commands_array(self):
        return self.commands_array

    
    def update(self, barcodes, barcodeData, center_x=None, frame_width=None):
        end_time = time.time()
        elapsed_time = end_time - self.start_time

        # If there are no commands left, stop the robot
        # Maybe make it so the state is either in SCANNING OR SPINNING
        if not self.commands_array and self.state == MovementState.WALKING_FORWARD and self.completed_maze:
            self.g_dog.stop()
            print("Maze Completed! Stopping Robot.")
            return #Just so that the robot doesn't do move or anything else that's unwarranted

        # The following state constantly checks whether or not a QR code hasn't been seen for a length of time and 
        # determines if it needs to handle 2 unique conditions or to keep walking forwards
        if self.state == MovementState.IDLE:
            if len(barcodes) == 0 and elapsed_time > 2:
                self.state = MovementState.NEARING_WALL
                print("I switched to NEARING_WALL")
                self.start_time = time.time()
            elif len(barcodes) != 0:
                self.g_dog.forward(5)
                self.start_time = time.time()

        # Continuation of IDLE, which determines if it has hit the wall or if it mistakenly didn't see the QR code
        # in view that it keeps walking forwards until it doesn't.
        elif self.state == MovementState.NEARING_WALL:
            if len(barcodes) != 0: #Meaning it has seen a QR Code
                self.state = MovementState.IDLE
                self.g_dog.forward(5)
                self.start_time = time.time()
            #The maximum tolerance of looking at the  wall and deciding what to do next.
            elif len(barcodes) == 0 and elapsed_time > 3.7:
                self.g_dog.stop()
                self.state = MovementState.SCANNING #So it goes to the wall and just turns without having to move back first
                self.start_time = time.time()

        #The initial state which allows the robot to find the initial QR Code
        elif self.state == MovementState.WALKING_FORWARD:
            if len(barcodes) != 0: #meaning that we now saw a QR Code
                self.g_dog.stop()
                self.state = MovementState.SCANNING
                self.current_node_location = barcodeData
                self.previous_node = self.current_node_location
            print("Scanning QR code")

        # The following handles the turning logic based on the next command in the Commands Array
        elif self.state == MovementState.SCANNING:
            #Ensures that the robot stops walking before executing the turn
            self.g_dog.stop()
            # The first barcode scanned determines our turn direction
            if self.target_command == "left":
                print("Target command is LEFT. Turning left.")
                self.g_dog.turnleft(1)
            elif self.target_command == "right":
                print("Target command is RIGHT. Turning right.")
                self.g_dog.turnright(1)
            self.state = MovementState.SPINNING
            self.adjacent_nodes.clear()  # Reset adjacent nodes
            print("Entering SPINNING state to detect new node.")

        elif self.state == MovementState.SPINNING:
            # Collect all QR codes seen during spinning
            if barcodeData and barcodeData != self.current_node_location:
                self.adjacent_nodes.add(barcodeData)

            # Stop when detecting the first new QR code that's aligned in the center of the Camera Display (within a margin of error)!!
            if center_x is not None and frame_width is not None:
                center_margin = 40  # pixels of acceptable offset from center
                frame_center = frame_width // 2
                
                if len(self.adjacent_nodes) > 0 and abs(center_x - frame_center) <= center_margin:
                    self.g_dog.stop()
                    self.current_node_location = list(self.adjacent_nodes)[0] # Take the first detected node
                    print(f"Detected new node: {self.current_node_location}, moving forward.")               
                    
                    # A second check for when the dog stops when it notices the QR code is centered.
                    # That checks whether the stopped dog's camera display is still within the center of the screen
                    if abs(center_x - frame_center) <= (center_margin-15):
                        self.g_dog.stop()
                        self.state = MovementState.ALIGNING_QR
                        self.start_time = time.time()
                        #Checking that even if it is empty, it still needs to execute the last command
                        #Thus, we need to keep the last command untouched, and pop() on an empty list throws error & problems
                        if not self.commands_array:
                            self.completed_maze = True
                        else:
                            self.commands_array.pop(0)  # Remove executed command
                            self.target_command = self.commands_array[0] if self.commands_array else None  # Update target command
                        self.adjacent_nodes.clear()
                    else:
                        if self.target_command == "left":
                            self.g_dog.turnright(1)
                        elif self.target_command == "right":
                            self.g_dog.turnleft(1)

        # This is for when we are at a node and want to recenter after seeing where we want to go!
        elif self.state == MovementState.ALIGNING_QR:
            if len(barcodes) > 0:
                barcode = barcodes[0]  # Just use the first one
                (x, y, w, h) = barcode.rect
                qr_center_x = x + w // 2
        
                frame_center_x = 640 // 2  # The Robot has 640x480 resolution
                margin = 15  # Pixel range for acceptable center alignment

                # The following statement determine if the displayed QR code is too far left, right, or centered
                if qr_center_x < frame_center_x - margin:
                    print("QR too far left. Re-aligning right.")
                    self.g_dog.turnright(1)
                elif qr_center_x > frame_center_x + margin:
                    print("QR too far right. Re-aligning left.")
                    self.g_dog.turnleft(1)
                else:
                    print("QR code is centered. Proceeding to IDLE.")
                    self.g_dog.stop()
                    self.g_dog.forward(5)
                    self.state = MovementState.IDLE
            else:
                print("Lost sight of QR during ALIGNING_QR. Returning to IDLE.")
                self.state = MovementState.IDLE
                self.g_dog.stop()

        #Fail Safe to make sure that the last_seen_nodes and Previous Nodes are being properally updated
        if len(barcodes) != 0:
            self.last_seen_node = self.previous_node
            # updates previous_node
            self.previous_node = barcodeData

In [11]:
#Acquiring the Initial Yaw of the Robot if the next team needs or wants to implement this feature
starting_yaw = g_dog.read_imu_raw()[8]
print("Initial yaw: ", starting_yaw)

Initial yaw:  -309.2986881510417


In [12]:
# Initial Start up of the Robot
g_dog.stop() #Ensures that it stopped all prevous functions of previous scripts

# Commands Maze Set up for the 2025 Expo by the Local Robots 2025 Capstone Team
commands_array = ["left", "right", "left", "left", "right", "left"]


# Initialize movement system
movement_system = MovementSystem(g_dog, commands_array)

def decodeDisplay(image, display):
    logging.debug('Barcode read')
    
    barcodes = pyzbar.decode(image)
    
    center_x = None
    if len(barcodes) == 0:
        barcodeData = None
        # Calling upon the main command for the Robot to autonomously navigate the maze
        movement_system.update(barcodes, barcodeData, center_x, display.shape[1])
    
    for barcode in barcodes:
        # 提取二维码的边界框的位置, 画出图像中条形码的边界框
        # Extract the position of the bounding box of the QR code, 
        # and draw the bounding box of the barcode in the image
        (x, y, w, h) = barcode.rect
        cv2.rectangle(display, (x, y), (x + w, y + h), (225, 225, 225), 2)
        # Acquiring the Center
        center_x = x + w // 2
        
        # 提取二维码数据为字节对象，转换成字符串
        # The QR code data is extracted as byte objects and converted into strings
        barcodeData = barcode.data.decode("utf-8")
        barcodeType = barcode.type

        # 绘出图像上条形码的数据和条形码类型  
        # Plot the barcode data and barcode type on the image
        text = "{} ({})".format(barcodeData, barcodeType)
        cv2.putText(display, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (225, 0, 0), 2)

        # Calling upon the main command for the Robot to autonomously navigate the maze
        movement_system.update(barcodes, barcodeData, center_x, display.shape[1])
    return display




# 检测二维码  detect qrcode
def Detect_Qrcode_Task():
    ret, frame = image.read()
    image_widget.value = bgr8_to_jpeg(frame)
    t_start = time.time()
    fps = 0
    while True:
        ret, frame = image.read()
        # 转为灰度图像  Convert to grayscale image
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = decodeDisplay(gray, frame)
        fps = fps + 1
        mfps = fps / (time.time() - t_start)
        cv2.putText(frame, "FPS " + str(int(mfps)), (40,40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 3)
        image_widget.value = bgr8_to_jpeg(frame)

I walked


In [13]:
# 启动摄像头显示任务  Start the camera display task
thread1 = threading.Thread(target=Detect_Qrcode_Task)
thread1.setDaemon(True)
thread1.start()

box_display = widgets.HBox([image_widget, button_Close_Camera])
display(box_display)

In [14]:
#g_dog.stop() #-> Just in case the robot won't stop#

In [16]:
_current_state = movement_system.get_state()
print(_current_state)

WALKING_FORWARD


In [17]:
_curr_location = movement_system.get_current_location()
_prev_node = movement_system.get_previous_node()
_bool_dermine_path = movement_system.get_bool_determine_path()
_target_command = movement_system.get_target_command()
_commands_array = movement_system.get_commands_array()

print("Current Node: ", _curr_location)
print("Previous Noe: ", _prev_node)
print("Bool Determine Path: ", _bool_dermine_path)
print("Target Command: ",_target_command)
print("Path to Take: ", _commands_array)

Current Node:  None
Previous Noe:  None
Bool Determine Path:  True
Target Command:  right
Path to Take:  ['right', 'left']


In [18]:
#This is the method of how we would print out IMU DATA
#while(True):
    #temp_imu = g_dog.read_imu_raw()[8]
    #print(temp_imu)